# Step 1: Postprocessing of raw eddy-covariance data
Example for basic post-processing of raw eddy-covariance measurements (from CSAT3/Campbell Scientific and Li-7200/LI-COR closed-path IR gas analyzer) including despiking, rotation, averaging and flagging. The notebooks (e.g., [02_basic-turbulence-diagnostics.ipynb](https://github.com/noctiluc3nt/ec_analyze/blob/main/notebooks/02_basic-turbulence-diagnostics.ipynb)) build upon this and allow for a more detailed analysis (e.g., [03_quadrant-analysis.ipynb](https://github.com/noctiluc3nt/ec_analyze/blob/main/notebooks/03_quadrant-analysis.ipynb)).

In [1]:
#loading Reddy package
install.packages("../src/Reddy_0.0.0.9000.tar.gz",repos=NULL,source=TRUE)
library(Reddy)

Installing package into ‘/home/lauracma/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)



In [2]:
#ec data files
dir_in="../data/ec-data_10Hz_raw"
files=list.files(dir_in,full.names=TRUE)
nf=length(files)

In [3]:
#allocate output
var_out=c("time","u_mean","v_mean","w_mean","ws_mean","wd_mean","T_mean","h2o_mean","co2_mean",
          "u_sd","v_sd","w_sd","T_sd","h2o_sd","co2_sd",
          "cov_uv","cov_uw","cov_vw","cov_wT","cov_h2ow","cov_co2w",
          "rot_angle1","rot_angle2","flag_stationarity","flag_w","flag_distortion")
nv=length(var_out)
dat=data.frame(array(NA,dim=c(nf,nv)))
colnames(dat)=var_out

In [4]:
#postprocessing per file (30 mins)
for (i in 1:nf) {
    tmp=read.table(files[i],sep=",",header=T)
    dat$time[i]=tmp$X[1]
    #despiking
    tmp$T_degC=despiking(tmp$T_degC,-40,30)
    tmp$u_m.s=despiking(tmp$u_m.s,-25,25)
    tmp$v_m.s=despiking(tmp$v_m.s,-25,25)
    tmp$w_m.s=despiking(tmp$w_m.s,-5,5)
    #wind before rotation
    dat$ws_mean[i]=sqrt(mean(tmp$u_m.s,na.rm=T)^2+mean(tmp$v_m.s,na.rm=T)^2)
    dat$wd_mean[i]=atan2(mean(tmp$v_m.s,na.rm=T),mean(tmp$u_m.s,na.rm=T))
    #rotation
    rot_wind=rotate_double(tmp$u_m.s,tmp$v_m.s,tmp$w_m.s)
    tmp$u_m.s=rot_wind$u
    tmp$v_m.s=rot_wind$v
    tmp$w_m.s=rot_wind$w
    dat$rot_angle1[i]=rot_wind$theta
    dat$rot_angle2[i]=rot_wind$phi
    #averaging
    dat$u_mean[i]=mean(tmp$u_m.s,na.rm=T)
    dat$v_mean[i]=mean(tmp$v_m.s,na.rm=T)
    dat$w_mean[i]=mean(tmp$w_m.s,na.rm=T)
    dat$T_mean[i]=mean(tmp$T_degC,na.rm=T)
    #unit conversion for gases
    tmp$rhoH2O=ppt2rho(tmp$H2O_ppt,dat$T_mean[i]+273.15,87000)
    tmp$rhoCO2=ppt2rho(tmp$CO2_ppm/1000,dat$T_mean[i]+273.15,87000,gas="CO2")
    dat$h2o_mean[i]=mean(tmp$rhoH2O,na.rm=T)
    dat$co2_mean[i]=mean(tmp$rhoCO2,na.rm=T)
    #sds
    dat$u_sd[i]=sd(tmp$u_m.s,na.rm=T)
    dat$v_sd[i]=sd(tmp$v_m.s,na.rm=T)
    dat$w_sd[i]=sd(tmp$w_m.s,na.rm=T)
    dat$T_sd[i]=sd(tmp$T_degC,na.rm=T)
    dat$h2o_sd[i]=sd(tmp$rhoH2O,na.rm=T)
    dat$co2_sd[i]=sd(tmp$rhoCO2,na.rm=T)
    #covs
    dat$cov_uw[i]=cov(tmp$u_m.s,tmp$w_m.s,use="pairwise.complete.obs")
    dat$cov_uv[i]=cov(tmp$u_m.s,tmp$v_m.s,use="pairwise.complete.obs")
    dat$cov_vw[i]=cov(tmp$v_m.s,tmp$w_m.s,use="pairwise.complete.obs")
    dat$cov_wT[i]=cov(tmp$T_degC,tmp$w_m.s,use="pairwise.complete.obs")
    dat$cov_h2ow[i]=cov(tmp$rhoH2O,tmp$w_m.s,use="pairwise.complete.obs")
    dat$cov_co2w[i]=cov(tmp$rhoCO2,tmp$w_m.s,use="pairwise.complete.obs")
    #SND correction
    #cov_wT_snd=SNDcorrection(tmp$u_m.s,tmp$v_m.s,tmp$w_m.s,tmp$T_degC)
    #flagging
    dat$flag_stationarity[i]=flag_stationarity(tmp$w_m.s,tmp$T_degC)
    dat$flag_w[i]=flag_w(dat$w_mean[i])
    dat$flag_distortion[i]=flag_distortion(tmp$u_m.s,tmp$v_m.s,dir_blocked=c(340,360))
}

In [5]:
#calculate fluxes
dat$sh=cov2sh(dat$cov_wT)
dat$lh=cov2lh(dat$cov_h2ow)

In [6]:
#look at output
dat

time,u_mean,v_mean,w_mean,ws_mean,wd_mean,T_mean,h2o_mean,co2_mean,u_sd,⋯,cov_wT,cov_h2ow,cov_co2w,rot_angle1,rot_angle2,flag_stationarity,flag_w,flag_distortion,sh,lh
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>
2018-07-20 08:30:00,2.872084,-3.939352e-16,6.931732e-06,2.873170,-2.289284,15.86938,0.007254132,0.0006009614,1.0621428,⋯,0.105285787,3.911379e-05,-1.567917e-07,-131.16630,0.34842842,1,0,NA,123.251207,103.108515
2018-07-20 09:00:00,2.864793,7.504684e-17,9.052527e-08,2.864538,-2.326045,16.55190,0.007584267,0.0005978138,1.1051216,⋯,0.160070314,6.168908e-05,-2.294700e-07,-133.27259,0.77454172,1,0,NA,187.383880,162.619600
2018-07-20 09:30:00,3.996526,9.365292e-16,3.049021e-05,4.002522,-2.035397,17.05704,0.007472065,0.0005954894,1.4094952,⋯,0.153351409,4.687620e-05,-1.862787e-07,-116.61967,0.28889573,1,0,NA,179.518496,123.571122
2018-07-20 10:00:00,4.998016,-2.812846e-16,-2.391239e-06,4.997530,-1.977737,17.60447,0.006762097,0.0005953881,1.2893259,⋯,0.155664878,4.682925e-05,-1.504328e-07,-113.31596,0.34831201,1,0,NA,182.226723,123.447352
2018-07-20 10:30:00,4.879095,4.062636e-16,2.187117e-05,4.880696,-2.014489,18.08994,0.005410862,0.0005948927,1.4699485,⋯,0.191448808,7.251737e-05,-1.989823e-07,-115.42174,0.65319222,1,0,NA,224.116637,191.164235
2018-07-20 11:00:00,5.225037,-8.977916e-17,-6.972624e-06,5.223984,-2.261205,18.24207,0.004422858,0.0005954766,1.3192267,⋯,0.149027117,5.722168e-05,-1.330638e-07,-129.55749,0.44710739,1,0,NA,174.456329,150.843015
2018-07-20 11:30:00,5.896774,-7.481298e-16,-4.307951e-05,5.893156,-2.141157,18.50137,0.003765156,0.0005955767,1.4571181,⋯,0.157559904,6.113976e-05,-1.312707e-07,-122.67927,0.81792595,1,0,NA,184.445107,161.171549
2018-07-20 12:00:00,5.211296,-3.062741e-16,-1.461607e-05,5.209715,-2.253118,18.57772,0.003418867,0.0005955199,1.2629639,⋯,0.147288599,6.336327e-05,-1.288697e-07,-129.09416,0.70850506,1,0,NA,172.421160,167.032959
2018-07-20 12:30:00,4.295926,-7.486225e-16,-1.727795e-06,4.295548,-2.194560,18.80679,0.003522845,0.0005946848,1.3898230,⋯,0.150429440,6.517965e-05,-1.343277e-07,-125.73902,0.55118076,1,0,NA,176.097937,171.821168


In [7]:
saveRDS(dat,file="../data/ec-data_30min_processed/processed_data_example.rds")

### Literature
- Foken, T. (2017). Micrometeorology, Springer-Verlag Berlin Heidelberg, DOI: 10.1007/978-3-642-25440-6.